In [ ]:
refined keywords = 'Gender','sex',

In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import requests
import re
import datetime

# Read in the main database
a1 = pd.read_csv("school_mainPA.csv")
a1 = a1[0:25]

# keywords = ["sexuality",'at birth','male puberty','Biological','Critical race theory',
#             'Partisan','Political','Indoctrination','Advocacy','Flag','Social policy','Race',
#             'Racial','Controversial','Profanity','Sexual conduct','Graphic violence','Age-inappropriate',
#             'Age-appropriate','Sexual acts','Sexualized','Nudity','LGBTQ','Gender ideology','sex assigned at birth',
#             '3101','6312(g)','Implied depictions of sexual acts',
#             'Partisan, Political, or Social Policy Advocacy', 'Neutrality']

keywords = ["gender theory",'holocaust denial','critical theory','at birth','male puberty','Critical race theory','Indoctrination','Social policy',
            'Racial','Controversial','Profanity','obscene','Graphic violence','transgender','cisgender',
            'Age-appropriate','Gender ideology','sex assigned at birth',
            '3101','6312(g)','Implied depictions of sexual acts','social justice',
            'Partisan, Political, or Social Policy Advocacy', 'Neutrality']

# DataFrame to store the extracted data
extracted_data = pd.DataFrame(columns=['School District', 'County', 'URL', 'Text Content', 'Keywords Found', 'Number of Keywords Found'])

# Get the current date for file naming
current_date = datetime.datetime.now().strftime("%Y-%m-%d")

# Loop through each row in the DataFrame
for index, row in a1.iterrows():
    district = row['school_dis']
    county = row['cty_name']
    url = row['base_link']

    try:
        driver = webdriver.Chrome()
        driver.get(url)
        time.sleep(1)
        element = driver.find_element(By.LINK_TEXT, "POLICIES")
        driver.execute_script('arguments[0].click()', element)
        time.sleep(5)
        soup_outer = BeautifulSoup(driver.page_source, "html.parser")
        href_elements = soup_outer.findChildren(attrs={'href': '#', 'unique': True})
        unique_values = [element.get("unique") for element in href_elements]
        time.sleep(2)
        for unique_value in unique_values:
            new_url = row['prePolicyID'] + unique_value
            try:
                response = requests.get(new_url)
                response.raise_for_status()  # Raise an exception if the response status code is not 200

                time.sleep(5)
                soup_inner = BeautifulSoup(response.content, "html.parser")
                container_div = soup_inner.find('div', id='policy-content')
                if container_div is not None:
                    text_content = container_div.get_text(" ", strip=True)
                    found_keywords = [keyword for keyword in keywords if re.search(r'\b{}\b'.format(re.escape(keyword)), text_content, re.IGNORECASE)]
                    if found_keywords:
                        extracted_data = extracted_data.append({'School District': district,
                                                                'County': county,
                                                                'URL': new_url,
                                                                'Keywords Found': found_keywords,
                                                                'Number of Keywords Found': len(found_keywords)}, ignore_index=True)
                else:
                    print("Container div not found for", new_url)
            except Exception as e:
                # Handle any exception
                print(f"Exception occurred: {str(e)}. Saving current data to CSV...")
                extracted_data.to_csv('extracted_data_crashedp1.csv', index=False)
                print("Data saved. Script crashed.")
            
            except requests.exceptions.RequestException as e:
                print(f"Failed to fetch data from {new_url}: {str(e)}")
            
            except KeyboardInterrupt:
                # Handle KeyboardInterrupt (Ctrl + C)
                print("KeyboardInterrupt detected. Saving current data to CSV...")
                extracted_data.to_csv('extracted_data_interruptedp1.csv', index=False)
                print("Data saved. Script interrupted.")

            time.sleep(2)

    except Exception as e:
        print(f"Exception occurred for base URL {url}: {str(e)}")

    finally:
        driver.quit()

# Save the extracted data to a CSV file
csv_filename = f"policy1_extracted_data_{current_date}.csv"
extracted_data.to_csv(csv_filename, index=False)
print(f"Extracted data saved to {csv_filename}")


Exception occurred: HTTPSConnectionPool(host='go.boarddocs.com', port=443): Max retries exceeded with url: /pa/bren/Board.nsf/goto?open&id=9X4P6A630F4D (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000170B19B6B80>: Failed to resolve 'go.boarddocs.com' ([Errno 11001] getaddrinfo failed)")). Saving current data to CSV...
Data saved. Script crashed.
Exception occurred: HTTPSConnectionPool(host='go.boarddocs.com', port=443): Max retries exceeded with url: /pa/bren/Board.nsf/goto?open&id=AK2SXY74E3B0 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000170B1989490>: Failed to resolve 'go.boarddocs.com' ([Errno 11001] getaddrinfo failed)")). Saving current data to CSV...
Data saved. Script crashed.
Container div not found for https://go.boarddocs.com/pa/carm/Board.nsf/goto?open&id=A8LV646A2E56
Extracted data saved to policy1_extracted_data_2023-08-15.csv
